# Libraries

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import gc
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from sklearn.model_selection import StratifiedGroupKFold

# Configurations

In [2]:
class CONFIG:
    target_col = "responder_6"
    lag_cols_original = ["date_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]
    lag_cols_rename = { f"responder_{idx}" : f"responder_{idx}_lag_1" for idx in range(9)}
    valid_ratio = 0.05
    start_dt = 1100
    chg_date_1 = 484
    chg_date_2 = 677
    chg_date_3 = 1530 # 마지막 partiton

# Load training data & select

In [3]:
# Use last 2 parquets

train = pl.scan_parquet(
    f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet"
).select(
    pl.int_range(pl.len(), dtype=pl.UInt32).alias("id"),
    pl.all(),
).with_columns(
    (pl.col(CONFIG.target_col)*2).cast(pl.Int32).alias("label"),
).filter(
    (pl.col("date_id") >= CONFIG.chg_date_1 - 30) & (pl.col("date_id") <= CONFIG.chg_date_1 + 30)
    | (pl.col("date_id") >= CONFIG.chg_date_2 - 30) & (pl.col("date_id") <= CONFIG.chg_date_2 + 30)
    | (pl.col("date_id") >= CONFIG.chg_date_3) # 데이터를 초반 두 번의 변화 두 번 전후 한달씩, 그리고 마지막 9 parquet을 가져옴.
).fill_null(
    strategy="forward"
).fill_null(
    value=0 # 결측치는 일단 단순하게 0으로 처리 
)

In [4]:
train = train.collect().to_pandas()
train.head()

,id,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,...,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,label
0,6655311,454,0,0,2.469772,2.011155,-1.183452,2.449150,1.968074,1.073371,...,-0.912550,-0.551103,0.071475,0.213802,0.920710,0.945452,0.355292,1.365892,2,1
1,6655312,454,0,1,4.609505,1.923074,-1.297473,1.898652,2.590320,0.983711,...,-0.381795,-0.265064,0.488816,0.821543,0.539706,0.821673,0.975165,1.707223,2,1
2,6655313,454,0,2,1.313092,2.103599,-0.866588,2.051928,2.211389,0.847589,...,-0.169111,-0.497740,2.245259,0.355446,0.867455,2.395301,0.417111,1.836413,2,4
3,6655314,454,0,3,0.953090,1.613543,-0.876514,2.681358,2.772389,1.059146,...,0.185034,0.866747,0.925349,-0.609420,1.878252,1.123068,-0.699646,2.347015,2,2
4,6655315,454,0,7,1.674723,2.253614,-1.061369,2.496380,2.403782,1.154420,...,-0.025160,-0.026998,0.154214,-0.225480,-0.242701,-0.107649,-0.422369,-0.321855,2,0


In [5]:
train = pl.DataFrame(train)

# Create Lags data from training data

In [6]:
lags = train.select(pl.col(CONFIG.lag_cols_original))
lags = lags.rename(CONFIG.lag_cols_rename)
lags = lags.with_columns(
    date_id = pl.col('date_id') + 1,  # lagged by 1 day
    )
lags = lags.group_by(["date_id", "symbol_id"], maintain_order=True).last()  # pick up last record of previous date
lags

date_id,symbol_id,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32
455,0,-0.37962,0.622232,-0.443895,-0.16413,-0.064856,-0.706662,0.023419,0.043093,0.062117
455,1,0.460542,0.481335,-0.318708,0.534235,0.308263,-0.149519,-0.001026,0.034362,-0.013399
455,2,-0.250504,0.560345,-0.099183,0.205178,0.185717,-0.502219,-0.080229,-0.005963,-0.165517
455,3,5.0,3.922242,4.143309,2.798591,1.049106,1.343205,-0.327025,-0.11451,-0.591432
455,7,-0.832327,0.260436,1.770283,0.370914,0.11369,-0.845703,0.087104,0.058233,0.175273
…,…,…,…,…,…,…,…,…,…,…
1699,34,0.243475,0.166927,0.38494,-0.174297,-0.066046,-0.038767,-0.132337,-0.022426,-0.252461
1699,35,0.850152,0.909382,1.015314,0.235962,0.122539,0.099559,-0.249584,-0.123571,-0.46063
1699,36,0.395684,-0.292574,-3.215846,-0.535129,-0.178484,-1.80815,-0.065355,-0.000367,-0.12517


# Merge training data and lags data

In [7]:
train = train.join(lags, on=["date_id", "symbol_id"],  how="left")
train

id,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,…,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,label,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
u32,i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32
6655311,454,0,0,2.469772,2.011155,-1.183452,2.44915,1.968074,1.073371,0.173038,0.764748,0.887245,-0.080057,11,7,76,-0.754221,0.611127,-0.62356,0.0,-0.413058,0.0,-1.452058,-1.056595,3.148603,0.0,0.578017,0.433606,1.267292,1.382232,0.0,0.0,0.445138,-0.228406,-0.699706,0.0,…,-0.158952,-0.326945,-0.212204,-1.547852,-1.304619,-0.828947,0.936439,-0.303425,-1.029249,0.165295,-0.575218,0.0,0.0,1.762617,2.387775,2.742359,2.962548,-0.599413,-0.91255,-0.551103,0.071475,0.213802,0.92071,0.945452,0.355292,1.365892,2,1,null,null,null,null,null,null,null,null,null
6655312,454,0,1,4.609505,1.923074,-1.297473,1.898652,2.59032,0.983711,0.124567,1.001583,0.855929,-0.097728,11,7,76,-1.01142,0.2557,-0.510994,0.0,-0.296151,0.0,-1.488241,-1.262973,2.169112,0.0,1.681371,1.543204,1.657605,2.555976,0.0,0.0,0.978464,-0.314007,-0.574282,0.0,…,-0.454976,-0.304905,-0.372808,-2.241868,-1.869178,-0.71385,-0.093584,-0.638239,-0.88669,0.706727,-0.525557,0.0,0.0,-0.067663,-0.052155,-0.205661,-0.232019,-0.450519,-0.381795,-0.265064,0.488816,0.821543,0.539706,0.821673,0.975165,1.707223,2,1,null,null,null,null,null,null,null,null,null
6655313,454,0,2,1.313092,2.103599,-0.866588,2.051928,2.211389,0.847589,0.177282,0.988273,0.435409,-0.178729,81,2,59,-0.766076,0.704434,-0.55336,0.0,-0.207497,0.0,-0.968711,-1.720343,-0.438561,0.0,-0.748295,-0.648906,0.808744,0.339712,0.0,0.0,0.20274,-0.517739,-0.732133,0.0,…,0.274716,2.711607,0.941928,-1.555242,-1.704168,-0.914428,1.449332,-0.201881,-0.799523,-0.21582,-0.561684,0.0,0.0,-0.349071,-0.428342,-0.289195,-0.362151,0.198349,-0.169111,-0.49774,2.245259,0.355446,0.867455,2.395301,0.417111,1.836413,2,4,null,null,null,null,null,null,null,null,null
6655314,454,0,3,0.95309,1.613543,-0.876514,2.681358,2.772389,1.059146,0.17529,0.9818,0.533703,-0.121689,4,3,11,-0.73376,2.234364,0.148274,0.0,-0.497003,0.0,-1.206971,-1.412848,1.289117,0.0,-1.145053,-1.662772,1.191818,1.489189,0.0,0.0,-2.443605,-0.554965,-0.79451,0.0,…,-0.225101,-0.251281,-0.028753,-1.555242,-1.704168,-0.636032,2.899746,0.391096,-1.116298,2.339685,-0.099127,0.0,0.0,-0.014938,-0.024851,-0.248691,-0.267081,0.095816,0.185034,0.866747,0.925349,-0.60942,1.878252,1.123068,-0.699646,2.347015,2,2,null,null,null,null,null,null,null,null,null
6655315,454,0,7,1.674723,2.253614,-1.061369,2.49638,2.403782,1.15442,0.144808,0.657552,1.034091,-0.138149,11,7,76,-1.104582,-0.237949,-0.581556,0.0,-0.046883,0.0,-0.936812,-1.082866,0.762833,0.0,-0.143877,1.309884,0.635429,0.155742,0.0,0.0,-0.813497,2.837898,1.569792,0.0,…,-0.298024,-0.548535,-0.117912,-1.568965,-1.734125,-1.062347,-0.066365,-0.734591,-0.909868,-0.368092,-0.830324,0.0,0.0,-0.395308,-0.328066,-0.282477,-0.342376,0.536572,-0.02516,-0.026998,0.154214,-0.22548,-0.242701,-0.107649,-0.422369,-0.321855,2,0,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

# Split training data and validation data

In [8]:
len_train   = train.select(pl.col("date_id")).shape[0]
valid_records = int(len_train * CONFIG.valid_ratio)
len_ofl_mdl = len_train - valid_records
last_tr_dt  = train.select(pl.col("date_id")).row(len_ofl_mdl)[0]

print(f"\n len_train = {len_train}")
print(f"\n len_ofl_mdl = {len_ofl_mdl}")
print(f"\n---> Last offline train date = {last_tr_dt}\n")

training_data = train.filter(pl.col("date_id").le(last_tr_dt))
validation_data = train.filter(pl.col("date_id").gt(last_tr_dt))


 len_train = 9101515

 len_ofl_mdl = 8646440

---> Last offline train date = 1686



In [9]:
validation_data

id,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,…,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,partition_id,label,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
u32,i16,i16,i8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i8,i8,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64,i32,f32,f32,f32,f32,f32,f32,f32,f32,f32
46675282,1687,0,0,3.781251,3.349843,0.963676,3.552205,3.070421,1.310815,-0.334233,0.29246,-0.463342,0.082186,11,7,76,-0.703837,1.303777,-0.395715,-0.744088,-0.417004,-0.717782,-0.790829,-1.892243,0.610748,-0.176413,1.30363,1.176901,2.203566,1.401162,0.750112,1.108462,0.688569,-0.6664,-0.904038,-0.184586,…,-0.495252,-0.408928,-0.505222,-0.772683,-2.835992,-1.026971,1.140901,-0.292416,-0.960177,0.324994,-0.588715,-0.261634,-0.263176,0.220839,0.265147,-0.179722,-0.12342,-0.388299,-0.033851,-0.340357,-0.077653,-0.971475,1.526969,0.01145,-1.353011,2.108581,9,0,-0.266812,-0.054878,-1.545247,0.385557,0.193606,-0.535236,0.058816,0.057375,0.083458
46675283,1687,0,1,2.606019,3.14255,1.244218,3.421399,3.048235,1.849931,-0.393432,0.321511,-0.522529,0.079715,11,7,76,-0.997407,1.887145,-0.087004,-0.744088,-0.359729,-0.717782,-1.231142,-1.691611,1.24168,-0.087754,0.883954,0.836346,1.564198,1.054291,-1.539081,-0.095403,1.502827,-0.415602,-0.640846,-0.058053,…,-0.319049,-0.324798,-0.524809,-1.333263,-2.538328,-0.782871,3.472205,0.283183,-0.763013,-0.081193,-0.704203,-0.261634,-0.263176,0.356346,0.375532,-0.123293,-0.143475,-0.332037,-0.512074,-0.418187,0.813138,-3.348194,1.64164,1.138862,-2.781623,3.425127,9,2,-0.241919,0.078192,-1.161707,0.000655,0.002046,-0.984262,-0.197686,-0.045213,-0.391194
46675284,1687,0,2,1.981104,2.78494,1.219115,2.595899,2.744964,1.372905,-0.565429,0.298982,-0.55758,0.028771,81,2,59,-0.538077,7.173572,1.052984,-0.744088,-0.204323,-0.717782,-1.146885,-2.23182,0.060473,-0.190955,1.217303,0.776203,0.596932,-0.477281,0.653693,0.649214,-0.156624,-0.592842,-0.842065,-0.229279,…,-0.181678,-0.297587,-0.319342,-1.502958,-1.685082,-0.280431,4.068408,0.735362,-0.836369,5.736065,0.943794,-0.261634,-0.263176,0.519323,0.493615,-0.083611,-0.103366,-0.343166,-0.26202,-0.408565,1.182349,2.643964,0.361312,1.431608,2.309818,0.698061,9,2,0.978288,0.76318,1.443481,-0.485256,-0.157901,0.084363,-0.164799,-0.109012,-0.762613
46675285,1687,0,3,2.110739,3.409801,0.865232,3.727028,3.588998,1.967243,-0.414937,0.336027,-0.631726,0.144795,4,3,11,-0.580152,3.140839,0.302595,-0.744088,-0.387682,-0.717782,-1.871841,-1.225321,-0.063711,-0.060089,0.55742,0.370213,0.762512,0.01872,-0.009819,-0.303811,-0.558923,-0.183521,-0.669897,-0.046879,…,-0.370689,-0.212923,-0.260396,-1.844798,-1.443868,-0.980988,2.543783,0.0621,-0.783769,3.202863,0.283404,-0.261634,-0.263176,0.668884,0.894722,0.027543,0.04745,-0.05093,-0.297161,-0.508374,-0.777918,-0.197728,-0.214127,-0.74216,-0.065933,-0.046738,9,-1,-0.855925,-0.629784,0.388898,-0.357745,-0.217354,0.082131,0.055058,0.040775,0.113845
46675286,1687,0,4,3.387619,3.457838,1.090773,3.220798,3.2206,1.926766,-0.267177,0.120857,-0.291993,0.044369,15,1,9,-1.218627,-0.002251,-0.476603,-0.744088,-0.56163,-0.717782,-0.856431,-1.271446,-1.845027,0.997514,1.072357,0.382435,4.423906,2.5854

# Save data as parquets

In [10]:
training_data.\
write_parquet(
    f"training_data.parquet"
)

In [11]:
validation_data.\
write_parquet(
    f"validation_data.parquet"
)